# FNIRSI DPS-150 Control Examples

Simple examples for controlling the DPS-150 power supply using the Python library.

In [1]:
import asyncio
from dps150 import DPS150, list_serial_ports
import time

## Device Connection

List available serial ports and connect to the device.

In [2]:
# List available ports
ports = list_serial_ports()
print("Available serial ports:")
for port in ports:
    print(f"  {port['device']}: {port['description']}")

Available serial ports:
  /dev/cu.debug-console: n/a
  /dev/cu.PigmentPete: n/a
  /dev/cu.Bluetooth-Incoming-Port: n/a
  /dev/cu.usbmodem1475E18C40251: AT32 Virtual Com Port


In [3]:
# Connect to device (change port as needed)
# On Linux/Mac: /dev/ttyUSB0, /dev/ttyACM0, etc.
# On Windows: COM3, COM4, etc.
PORT = "/dev/cu.usbmodem1475E18C40251"  # Change this to your port

# Connect to device (change port as needed)
# On Linux/Mac: /dev/ttyUSB0, /dev/ttyACM0, etc.
# On Windows: COM3, COM4, etc.
PORT = "/dev/cu.usbmodem1475E18C40251"  # Change this to your port

device = DPS150(port=PORT)
await device.connect()

# Get device info
info = await device.get_info()
print(f"Connected to: {info.model_name}")
print(f"Hardware: {info.hardware_version}")
print(f"Firmware: {info.firmware_version}")

Connected to: DPS-150
Hardware: V1.0
Firmware: V1.2


## Basic Control Examples

In [4]:
# Example 1: Set voltage and current, then enable output
print("Example 1: Basic voltage and current control")
await device.set_voltage(12.0)
await device.set_current(1.0)
await device.enable_output()

# Get current state
state = await device.get_all()
print(f"Set: {state.set_voltage:.2f}V @ {state.set_current:.2f}A")
print(f"Output: {state.output_voltage:.3f}V @ {state.output_current:.3f}A, {state.output_power:.3f}W")
print()

Example 1: Basic voltage and current control
Set: 12.00V @ 1.00A
Output: 0.400V @ 0.001A, 0.000W



In [7]:
# Example 2: Voltage sweep
print("Example 2: Voltage sweep from 0V to 12V")
await device.set_current(1.0)  # Set current limit
await device.enable_output()

# Sweep voltage from 0 to 12V in 1V steps
for voltage in range(0, 13):
    await device.set_voltage(float(voltage))
    await asyncio.sleep(0.5)  # Wait 0.5 seconds at each voltage
    state = await device.get_all()
    print(f"Set: {voltage}V -> Output: {state.output_voltage:.3f}V, "
          f"{state.output_current:.3f}A, {state.output_power:.3f}W")

await device.disable_output()
print("Sweep complete\n")

Example 2: Voltage sweep from 0V to 12V
Set: 0V -> Output: 0.000V, 0.000A, 0.000W
Set: 1V -> Output: 0.891V, 0.000A, 0.000W
Set: 2V -> Output: 1.886V, 0.000A, 0.000W
Set: 3V -> Output: 2.896V, 0.000A, 0.000W
Set: 4V -> Output: 3.887V, 0.000A, 0.000W
Set: 5V -> Output: 4.884V, 0.000A, 0.000W
Set: 6V -> Output: 5.192V, 0.000A, 0.000W
Set: 7V -> Output: 0.208V, 0.000A, 0.000W
Set: 8V -> Output: 0.000V, 0.000A, 0.000W
Set: 9V -> Output: 0.000V, 0.000A, 0.000W
Set: 10V -> Output: 0.000V, 0.000A, 0.000W
Set: 11V -> Output: 0.000V, 0.000A, 0.000W
Set: 12V -> Output: 0.000V, 0.000A, 0.000W
Sweep complete



In [8]:
# Example 3: Fine voltage sweep with smaller steps
print("Example 3: Fine voltage sweep (0-5V in 0.1V steps)")
await device.set_current(0.5)
await device.enable_output()

voltages = []
currents = []
powers = []

for v in range(0, 51):  # 0 to 5.0V in 0.1V steps
    voltage = v / 10.0
    await device.set_voltage(voltage)
    await asyncio.sleep(0.2)  # Wait 0.2 seconds
    state = await device.get_all()
    voltages.append(state.output_voltage)
    currents.append(state.output_current)
    powers.append(state.output_power)
    if v % 10 == 0:  # Print every 1V
        print(f"{voltage:.1f}V: {state.output_voltage:.3f}V, {state.output_current:.3f}A, {state.output_power:.3f}W")

await device.disable_output()
print("Fine sweep complete\n")

Example 3: Fine voltage sweep (0-5V in 0.1V steps)
0.0V: 0.000V, 0.000A, 0.000W
1.0V: 0.922V, 0.000A, 0.000W
2.0V: 1.922V, 0.000A, 0.000W
3.0V: 2.918V, 0.000A, 0.000W
4.0V: 3.926V, 0.000A, 0.000W
5.0V: 4.922V, 0.000A, 0.000W
Fine sweep complete



In [9]:
# Example 4: Current sweep
print("Example 4: Current sweep from 0A to 2A")
await device.set_voltage(5.0)  # Set voltage
await device.enable_output()

# Sweep current from 0 to 2A in 0.1A steps
for i in range(0, 21):
    current = i / 10.0
    await device.set_current(current)
    await asyncio.sleep(0.3)
    state = await device.get_all()
    print(f"Set: {current:.1f}A -> Output: {state.output_voltage:.3f}V, "
          f"{state.output_current:.3f}A, {state.output_power:.3f}W")

await device.disable_output()
print("Current sweep complete\n")

Example 4: Current sweep from 0A to 2A
Set: 0.0A -> Output: 3.283V, 0.001A, 0.003W
Set: 0.1A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.2A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.3A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.4A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.5A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.6A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.7A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.8A -> Output: 5.000V, 0.000A, 0.000W
Set: 0.9A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.0A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.1A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.2A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.3A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.4A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.5A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.6A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.7A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.8A -> Output: 5.000V, 0.000A, 0.000W
Set: 1.9A -> Output: 5.000V, 0.000A, 0.000W
Set: 2.0A -> Output: 5.000V, 0.000A, 0.000W
Current sweep complete



In [7]:
# Example 5: Monitor output over time
print("Example 5: Monitor output for 10 seconds")
await device.set_voltage(10.0)
await device.set_current(1.0)
await device.enable_output()

for i in range(10):
    state = await device.get_all()
    print(f"Time: {i}s - V: {state.output_voltage:.3f}V, "
          f"I: {state.output_current:.3f}A, P: {state.output_power:.3f}W, "
          f"Temp: {state.temperature:.1f}°C")
    await asyncio.sleep(1)

await device.disable_output()
print("Monitoring complete\n")

Example 5: Monitor output for 10 seconds
Time: 0s - V: 0.403V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 1s - V: 5.195V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 2s - V: 5.195V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 3s - V: 5.196V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 4s - V: 5.196V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 5s - V: 5.197V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 6s - V: 5.198V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 7s - V: 5.198V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 8s - V: 5.198V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Time: 9s - V: 5.197V, I: 0.000A, P: 0.000W, Temp: 30.0°C
Monitoring complete



In [6]:
# Example 7: Using preset groups
print("Example 7: Using preset groups")
# Set group 1
await device.set_group(1, 5.0, 0.5)
print("Group 1 set: 5.0V @ 0.5A")

# Set group 2
await device.set_group(2, 12.0, 1.0)
print("Group 2 set: 12.0V @ 1.0A")

# Load and use group 1
await device.load_group(1)
await device.enable_output()
await asyncio.sleep(2)
state = await device.get_all()
print(f"Group 1 active: {state.output_voltage:.3f}V @ {state.output_current:.3f}A")

# Switch to group 2
await device.load_group(2)
await asyncio.sleep(2)
state = await device.get_all()
print(f"Group 2 active: {state.output_voltage:.3f}V @ {state.output_current:.3f}A")

await device.disable_output()
print("Preset groups example complete\n")

Example 7: Using preset groups
Group 1 set: 5.0V @ 0.5A
Group 2 set: 12.0V @ 1.0A
Group 1 active: 5.000V @ 0.000A
Group 2 active: 5.195V @ 0.000A
Preset groups example complete



In [5]:
# Example 6: Setting protection limits
print("Example 6: Setting overvoltage, overcurrent, and over-power protection")
print("Getting current protection settings...")
state = await device.get_all()
print(f"Current OVP: {state.over_voltage_protection:.2f}V")
print(f"Current OCP: {state.over_current_protection:.2f}A")
print(f"Current OPP: {state.over_power_protection:.2f}W")
print()

# Set protection limits
print("Setting protection limits...")
await device.set_ovp(15.0)  # Over-voltage protection at 15V
await device.set_ocp(2.0)   # Over-current protection at 2A
await device.set_opp(20.0)  # Over-power protection at 20W
print("Protection limits set:")
print("  OVP: 15.0V")
print("  OCP: 2.0A")
print("  OPP: 20.0W")
print()

# Verify the settings
await asyncio.sleep(0.5)
state = await device.get_all()
print("Verified protection settings:")
print(f"  OVP: {state.over_voltage_protection:.2f}V")
print(f"  OCP: {state.over_current_protection:.2f}A")
print(f"  OPP: {state.over_power_protection:.2f}W")
print("Protection settings example complete\n")

Example 6: Setting overvoltage, overcurrent, and over-power protection
Getting current protection settings...
Current OVP: 5.00V
Current OCP: 1.00A
Current OPP: 150.00W

Setting protection limits...
Protection limits set:
  OVP: 15.0V
  OCP: 2.0A
  OPP: 20.0W

Verified protection settings:
  OVP: 15.00V
  OCP: 2.00A
  OPP: 20.00W
Protection settings example complete



## Cleanup

Disconnect from the device when done:

In [12]:
# Disconnect from device
await device.disable_output()
await device.disconnect()
print("Disconnected from device")

Disconnected from device
